In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math, json
import torch
from sentence_transformers import SentenceTransformer

In [115]:
torch.__version__

'2.2.2'

In [187]:
srt_sentences = []
with open("NUS-FutureWork-first10-modified-copy.srt") as f:
    srt_sentences = f.readlines()

In [117]:
def checknum(a):
    try:
        int(a)
    except:
        return False
    return True

In [188]:
filtered_srt_sentences = []
for l in srt_sentences:
    l_ = l.replace('\n','')
    if l_ == '':
        continue 
    
    if '-->' not in l_:
        if checknum(l_):
            continue
        filtered_srt_sentences.append(l_)
        
final_script = ' '.join(filtered_srt_sentences)

In [189]:
final_script

"Ladies and gentlemen. Please put your hand to gather to welcome all guest of owner Minister for education Mr. Chan Chun Seang A very good morning to gasthof owner Minister for education, Mr. Chan Chee Seng, Keynote Speaker Prof James Lander and Yours President Prof Tan Eng Chai, Deputy President of Academic Affairs and Provost, Prof Aren't and Distinguished Guest, Alumni and Partners Here are Nurses and tuning in online Welcome to future works to point to the second edition of the nus lifelong learning festival. My name is jawi and a proud ulama staff member of nus and ummc for this morning. The theme for this years conference is a infection and we are grateful that so many industry leaders and experts have the general sale of their time today to share their invaluable experiences in this area. We are so grateful that you have taken the time to attend the conference and we hope that you will live with the insights connections and inspiration for our future wife. I. Before we begin a q

In [190]:
final_sentences = []

for sentence in final_script.split("."):
    final_sentences.append(sentence.strip()+".")

In [191]:
final_sentences

['Ladies and gentlemen.',
 'Please put your hand to gather to welcome all guest of owner Minister for education Mr.',
 'Chan Chun Seang A very good morning to gasthof owner Minister for education, Mr.',
 "Chan Chee Seng, Keynote Speaker Prof James Lander and Yours President Prof Tan Eng Chai, Deputy President of Academic Affairs and Provost, Prof Aren't and Distinguished Guest, Alumni and Partners Here are Nurses and tuning in online Welcome to future works to point to the second edition of the nus lifelong learning festival.",
 'My name is jawi and a proud ulama staff member of nus and ummc for this morning.',
 'The theme for this years conference is a infection and we are grateful that so many industry leaders and experts have the general sale of their time today to share their invaluable experiences in this area.',
 'We are so grateful that you have taken the time to attend the conference and we hope that you will live with the insights connections and inspiration for our future wif

# Summarize this transcript "&lt;srt file contents &gt;".  In the summary, use very short sentences and do not use abbreviations and titles

In [66]:
# chat_gpt_summary =  "The transcript is from the opening of an event, likely a conference, where the host introduces key guests, "+\
# "including the Minister for Education, Chan Chun Seng, and other distinguished individuals like keynote speaker James Lander "+\
# "and National University of Singapore President Tan Eng Chai. "+\
# "The event is the second edition of the National University of Singapore Lifelong Learning Festival, with a focus on 'Artificial Intelligence in Action.' "+\
# "The National University of Singapore President emphasizes the transformative power of artificial intelligence, comparing it to the advent of personal  "+\
# "computers and the internet. He discusses the university's efforts in artificial intelligence research, including the establishment of an  "+\
# "artificial intelligence institute aimed at accelerating research and promoting real-world applications. The President also highlights initiatives  "+\
# "to integrate artificial intelligence into teaching, learning, and university operations, including staff training in artificial intelligence and  "+\
# "data literacy. The event is positioned as a critical moment in the artificial intelligence revolution, urging participants to embrace and innovate  "+\
# "with new technologies."

chat_gpt_summary = "The speaker welcomes the audience. "+\
"The event is the NUS Lifelong Learning Festival. "+\
"The theme is AI in action. The focus is on how AI is transforming industries and work. "+\
"NUS has started an AI institute to advance research and partnerships. The institute also addresses "+\
"ethical concerns. NUS is using AI in teaching, learning, and administration. "+\
"Workshops guide staff and students on AI use. Around 5,000 staff completed courses in data literacy and AI"

In [170]:
chatgpt_summary_sentences = []

for sentence in chat_gpt_summary.split("."):
    chatgpt_summary_sentences.append(sentence.strip()+".")

In [171]:
chatgpt_summary_sentences

['The speaker welcomes the audience.',
 'The event is the NUS Lifelong Learning Festival.',
 'The theme is AI in action.',
 'The focus is on how AI is transforming industries and work.',
 'NUS has started an AI institute to advance research and partnerships.',
 'The institute also addresses ethical concerns.',
 'NUS is using AI in teaching, learning, and administration.',
 'Workshops guide staff and students on AI use.',
 'Around 5,000 staff completed courses in data literacy and AI.']

In [200]:
# Find time lengths

with open("NUS-FutureWork-first10.json") as f:
    speechmatic_json = json.load(f)
f.close()

In [201]:

# # One way - Needs more work

# time_segs = []
# for word_ind, word in enumerate(speechmatic_json['results']):
#     if 'is_eos' in word and word['is_eos']:
#         time_segs.append(word['end_time'] - word['start_time'])
# len(time_segs), len(final_sentences)

In [216]:
start_times = []
end_times = []
time_segs = []
final_timed_sentences = []
cnt = 0
for ind, final_sentence in enumerate(final_sentences):
    final_sentence_tokens = []
    start_time = 0
    end_time = 0
    for final_sentence_token in final_sentence.split(' '):
        if final_sentence_token.endswith(','):
            final_sentence_tokens.append(final_sentence_token.replace(',',''))
            final_sentence_tokens.append(',') #separate comma token
        else:
            final_sentence_tokens.append(final_sentence_token)
        
    # for tk in final_sentence_tokens:
    if len(final_sentence_tokens) > 5:
        print(final_sentence_tokens[0], final_sentence_tokens[1], final_sentence_tokens[2], final_sentence_tokens[3], final_sentence_tokens[4],'|', final_sentence_tokens[-1].split('.')[0])
        for word_ind, word in enumerate(speechmatic_json['results']):
            if word['alternatives'][0]['content'].lower() == final_sentence_tokens[0].lower() and \
                speechmatic_json['results'][word_ind+1]['alternatives'][0]['content'].lower() == final_sentence_tokens[1].lower() and \
                speechmatic_json['results'][word_ind+2]['alternatives'][0]['content'].lower() == final_sentence_tokens[2].lower() and \
                speechmatic_json['results'][word_ind+3]['alternatives'][0]['content'].lower() == final_sentence_tokens[3].lower() and \
                speechmatic_json['results'][word_ind+4]['alternatives'][0]['content'].lower() == final_sentence_tokens[4].lower():
                print(word['start_time'], word['end_time'])
                time_segs.append(word['end_time'] - word['start_time'])
                final_timed_sentences.append(final_sentence)
                start_times.append(word['start_time'])
                end_times.append(word['end_time'])
        print(ind, '----')
len(time_segs), len(final_timed_sentences)        

Please put your hand to | Mr
9.83 10.31
1 ----
Chan Chun Seang A very | Mr
15.89 16.28
2 ----
Chan Chee Seng , Keynote | festival
68.47 68.71
3 ----
My name is jawi and | morning
92.59 92.8
4 ----
The theme for this years | area
99.37 99.52
5 ----
We are so grateful that | wife
113.59 113.77
6 ----
Before we begin a quick | briefing
124.72 125.05
8 ----
In case of emergency Please | state
128.71 128.86
9 ----
Or at the back of | instructions
134.32 134.47
10 ----
We would like to invite | Thanks
144.04 144.13
12 ----
Chan Chuncheon , the Guest | Friends
168.4 168.64
14 ----
Good morning and I bet | table
173.44 173.62
15 ----
The theme for this year's | action
184.14 184.29
16 ----
Many of us have already | action
195.58 195.88
18 ----
We will have the privilege | deploy
213.97 214.18
20 ----
The changes are as broad | journey
227.29 227.47
21 ----
We are in the watershed | revolution
237.76 237.91
22 ----
Some of US here to | period
244.57 244.93
23 ----
Nearly forty years ago where |

(34, 34)

In [217]:
for sent_ind, sent in enumerate(final_timed_sentences):
    print(time_segs[sent_ind], sent)

0.4800000000000004 Please put your hand to gather to welcome all guest of owner Minister for education Mr.
0.39000000000000057 Chan Chun Seang A very good morning to gasthof owner Minister for education, Mr.
0.23999999999999488 Chan Chee Seng, Keynote Speaker Prof James Lander and Yours President Prof Tan Eng Chai, Deputy President of Academic Affairs and Provost, Prof Aren't and Distinguished Guest, Alumni and Partners Here are Nurses and tuning in online Welcome to future works to point to the second edition of the nus lifelong learning festival.
0.20999999999999375 My name is jawi and a proud ulama staff member of nus and ummc for this morning.
0.14999999999999147 The theme for this years conference is a infection and we are grateful that so many industry leaders and experts have the general sale of their time today to share their invaluable experiences in this area.
0.1799999999999926 We are so grateful that you have taken the time to attend the conference and we hope that you will

In [59]:
# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

In [210]:
chatgpt_embeddings = model.encode(chatgpt_summary_sentences)
final_sentences_embeddings = model.encode(final_sentences)

In [211]:
similarities = model.similarity(chatgpt_embeddings, final_sentences_embeddings) # Not used


cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
cos_similarities = []
for chatgpt_embedding in chatgpt_embeddings:
    chatgpt_sim = []
    for final_sentences_embedding in final_sentences_embeddings:
        chatgpt_sim.append(cos(torch.from_numpy(chatgpt_embedding), torch.from_numpy(final_sentences_embedding)).numpy())
    cos_similarities.append(chatgpt_sim)

In [212]:
for i in range(len(chatgpt_summary_sentences)):
    print(chatgpt_summary_sentences[i], '==>', final_sentences[np.argmax(cos_similarities[i])])
    print('--------')

The speaker welcomes the audience. ==> We would like to invite NS President Prof Tan Eng Chai to give the Welcome Actress Thanks.
--------
The event is the NUS Lifelong Learning Festival. ==> The theme for this year's learning Festival is future work to point zero i in action.
--------
The theme is AI in action. ==> We are actively positioning ourselves to right the potential of ai On the Research Front.
--------
The focus is on how AI is transforming industries and work. ==> The changes are as broad about and how They are bringing the workforce on this new ai journey.
--------
NUS has started an AI institute to advance research and partnerships. ==> We are actively positioning ourselves to right the potential of ai On the Research Front.
--------
The institute also addresses ethical concerns. ==> This institute will also conduct research on how to regulate And to safeguard Transparency and Accountability.
--------
NUS is using AI in teaching, learning, and administration. ==> For exam

In [195]:
final_sentences

['Ladies and gentlemen.',
 'Please put your hand to gather to welcome all guest of owner Minister for education Mr.',
 'Chan Chun Seang A very good morning to gasthof owner Minister for education, Mr.',
 "Chan Chee Seng, Keynote Speaker Prof James Lander and Yours President Prof Tan Eng Chai, Deputy President of Academic Affairs and Provost, Prof Aren't and Distinguished Guest, Alumni and Partners Here are Nurses and tuning in online Welcome to future works to point to the second edition of the nus lifelong learning festival.",
 'My name is jawi and a proud ulama staff member of nus and ummc for this morning.',
 'The theme for this years conference is a infection and we are grateful that so many industry leaders and experts have the general sale of their time today to share their invaluable experiences in this area.',
 'We are so grateful that you have taken the time to attend the conference and we hope that you will live with the insights connections and inspiration for our future wif

In [83]:
len(speechmatic_json['results'])

959